## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from functools import reduce
import random

## Utils

#### Test function

In [2]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [3]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


## Train data

In [4]:
def shuffle_df(data):
    for index, row in data.iterrows():
        rank = -1
    
        row['S1'] = random.randint(1,4)
        row['S2'] = random.randint(1,4)
        row['S3'] = random.randint(1,4)
        row['S4'] = random.randint(1,4)
        row['S5'] = random.randint(1,4)
        row['C1'] = random.randint(1,13)
        row['C2'] = random.randint(1,13)
        row['C3'] = random.randint(1,13)
        row['C4'] = random.randint(1,13)
        row['C5'] = random.randint(1,13)
        row['hand'] = calc_hand_label(row[['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5']])
    return data

In [5]:
suits = ['S1','S2','S3','S4','S5']
cards = ['C1','C2','C3','C4','C5']
distances = ['D1', 'D2', 'D3', 'D4', 'D5']

In [6]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [7]:
train_dataset = train_dataset.append(shuffle_df(train_dataset),ignore_index=True).append(shuffle_df(train_dataset),ignore_index=True).append(shuffle_df(train_dataset),ignore_index=True)

In [8]:
train_dataset

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,2,13,2,11,2,4,1,11,2,12,1
1,3,12,3,8,2,6,2,1,2,8,1
2,3,4,4,2,2,10,3,13,4,10,1
3,4,13,2,3,2,5,4,4,2,8,0
4,4,7,2,9,1,11,3,5,2,4,0
5,4,11,1,4,3,1,1,10,1,6,0
6,1,11,2,7,1,12,3,13,2,5,0
7,4,8,1,9,1,3,3,4,2,10,0
8,2,1,1,10,3,12,2,9,4,1,1
9,4,2,2,11,4,2,4,1,2,4,1


In [10]:
data = train_dataset.drop('hand', axis=1)
train_labels = train_dataset['hand']

In [11]:
labels_vect = tf.keras.utils.to_categorical(train_labels)

#### Adding distance features

In [12]:
def get_distances(hand):
    distances_list = []
    for i, card in enumerate(hand):
        if (i < len(hand) - 1):
            distances_list.append(hand[i+1] - card)
        else:
            distances_list.append(13 - card + hand[0])
    return pd.Series(distances_list)

def with_distances(data):
    sorted = data[cards]
    sorted.values.sort()

    distances_df = sorted.apply(get_distances, axis=1)
    distances_df.columns = distances
    
    data_with_distances = pd.concat([data, distances_df], axis=1)
    return data_with_distances

In [13]:
data_with_distances = with_distances(data)
data_with_distances.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,D1,D2,D3,D4,D5
0,2,13,2,11,2,4,1,11,2,12,7,0,1,1,4
1,3,12,3,8,2,6,2,1,2,8,5,2,0,4,2
2,3,4,4,2,2,10,3,13,4,10,2,6,0,3,2
3,4,13,2,3,2,5,4,4,2,8,1,1,3,5,3
4,4,7,2,9,1,11,3,5,2,4,1,2,2,2,6


#### One hot encoding

In [14]:
def one_hot_encoded(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

In [15]:
encoder = OneHotEncoder()
data_encoded = one_hot_encoded(data_with_distances, suits, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(100040, 30)


,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,...,10,11,12,13,14,15,16,17,18,19
0,13,11,4,11,12,7,0,1,1,4,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,12,8,6,1,8,5,2,0,4,2,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,4,2,10,13,10,2,6,0,3,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,13,3,5,4,8,1,1,3,5,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,7,9,11,5,4,1,2,2,2,6,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [16]:
def normalized(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data_normalized, data.drop(columns, axis=1)], axis=1)
    return to_return

In [17]:
mean_c = data_encoded[cards].mean().mean()
std_c = pd.Series(data_encoded[cards].values.flatten()).std()
mean_d = data_encoded[distances].mean().mean()
std_d = pd.Series(data_encoded[distances].values.flatten()).std()

data_normalized = normalized(data_encoded, cards, mean_c, std_c)
data_normalized = normalized(data_normalized, distances, mean_d, std_d)
data_normalized.head()

,D1,D2,D3,D4,D5,C1,C2,C3,C4,C5,...,10,11,12,13,14,15,16,17,18,19
0,2.036898,-1.203622,-0.740690,-0.740690,0.648104,1.604573,1.070199,-0.800109,1.070199,1.337386,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.111036,-0.277759,-1.203622,0.648104,-0.277759,1.337386,0.268639,-0.265735,-1.601669,0.268639,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.277759,1.573967,-1.203622,0.185173,-0.277759,-0.800109,-1.334482,0.803012,1.604573,0.803012,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,-0.740690,-0.740690,0.185173,1.111036,0.185173,1.604573,-1.067295,-0.532922,-0.800109,0.268639,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,-0.740690,-0.277759,-0.277759,-0.277759,1.573967,0.001452,0.535826,1.070199,-0.532922,-0.800109,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [18]:
data_normalized.shape

(100040, 30)

## Neural Network

In [24]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(120, activation='relu', input_shape=(30,)))
model.add(keras.layers.Dropout(0.2))
#hidden
model.add(keras.layers.Dense(240, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dense(60, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=500, batch_size=512)

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 32us/step - loss: 1.1389 - acc: 0.5216

Epoch 2/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 29us/step - loss: 0.7127 - acc: 0.7077

Epoch 3/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 27us/step - loss: 0.3386 - acc: 0.8803

Epoch 4/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 0.1152 - acc: 0.9660

Epoch 5/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 23us/step - loss: 0.0585 - acc: 0.9852

Epoch 6/500
100040/100040 [==============================]100040/100040 [==============================]

100040/100040 [==============================]100040/100040 [==============================] - 3s 30us/step - loss: 0.0035 - acc: 0.9992

Epoch 52/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 27us/step - loss: 0.0024 - acc: 0.9994

Epoch 53/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 0.0021 - acc: 0.9995

Epoch 54/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 27us/step - loss: 0.0023 - acc: 0.9995

Epoch 55/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 0.0024 - acc: 0.9995

Epoch 56/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 25us/step - loss: 0.0021 - acc: 0.9996

Epoch 57/500
100040/100040 [==============================]100040/100040 [==============================] -

100040/100040 [==============================]100040/100040 [==============================] - 3s 27us/step - loss: 0.0017 - acc: 0.9997

Epoch 106/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 27us/step - loss: 0.0014 - acc: 0.9998

Epoch 107/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 27us/step - loss: 0.0014 - acc: 0.9998

Epoch 108/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 26us/step - loss: 0.0018 - acc: 0.9997

Epoch 109/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 26us/step - loss: 0.0014 - acc: 0.9998

Epoch 110/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 26us/step - loss: 0.0014 - acc: 0.9998

Epoch 111/500
100040/100040 [==============================]100040/100040 [===========================

100040/100040 [==============================]100040/100040 [==============================] - 3s 32us/step - loss: 0.0012 - acc: 0.9998

Epoch 159/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 32us/step - loss: 0.0010 - acc: 0.9999

Epoch 160/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 32us/step - loss: 0.0014 - acc: 0.9998

Epoch 161/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 26us/step - loss: 0.0013 - acc: 0.9998

Epoch 162/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 28us/step - loss: 0.0012 - acc: 0.9998

Epoch 163/500
100040/100040 [==============================]100040/100040 [==============================] - 4s 35us/step - loss: 0.0017 - acc: 0.9998

Epoch 164/500
100040/100040 [==============================]100040/100040 [===========================

...

100040/100040 [==============================]100040/100040 [==============================] - 2s 22us/step - loss: 8.5887e-04 - acc: 0.9999

Epoch 265/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 23us/step - loss: 0.0012 - acc: 0.9999

Epoch 266/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 23us/step - loss: 0.0012 - acc: 0.9999

Epoch 267/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 22us/step - loss: 0.0011 - acc: 0.9999

Epoch 268/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 23us/step - loss: 0.0010 - acc: 0.9999

Epoch 269/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 24us/step - loss: 0.0012 - acc: 0.9999

Epoch 270/500
100040/100040 [==============================]100040/100040 [=======================

100040/100040 [==============================]100040/100040 [==============================] - 2s 23us/step - loss: 0.0010 - acc: 0.9999

Epoch 318/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 23us/step - loss: 0.0010 - acc: 0.9999

Epoch 319/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 24us/step - loss: 9.4064e-04 - acc: 0.9999

Epoch 320/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 30us/step - loss: 8.4144e-04 - acc: 0.9999

Epoch 321/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 23us/step - loss: 9.7253e-04 - acc: 0.9999

Epoch 322/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 23us/step - loss: 0.0012 - acc: 0.9999

Epoch 323/500
100040/100040 [==============================]100040/100040 [===============

100040/100040 [==============================]100040/100040 [==============================] - 2s 22us/step - loss: 0.0011 - acc: 0.9999

Epoch 371/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 22us/step - loss: 0.0010 - acc: 0.9999

Epoch 372/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 22us/step - loss: 0.0014 - acc: 0.9999

Epoch 373/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 22us/step - loss: 0.0013 - acc: 0.9998

Epoch 374/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 22us/step - loss: 9.3301e-04 - acc: 0.9999

Epoch 375/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 23us/step - loss: 9.3072e-04 - acc: 0.9999

Epoch 376/500
100040/100040 [==============================]100040/100040 [===================

100040/100040 [==============================]100040/100040 [==============================] - 3s 27us/step - loss: 9.1742e-04 - acc: 0.9999

Epoch 424/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 28us/step - loss: 9.8488e-04 - acc: 0.9999

Epoch 425/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 26us/step - loss: 9.7069e-04 - acc: 0.9999

Epoch 426/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 26us/step - loss: 9.1367e-04 - acc: 0.9999

Epoch 427/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 8.9858e-04 - acc: 0.9999

Epoch 428/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 0.0012 - acc: 0.9999

Epoch 429/500
100040/100040 [==============================]100040/100040 [=======

100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 9.3661e-04 - acc: 0.9999

Epoch 477/500
100040/100040 [==============================]100040/100040 [==============================] - 2s 25us/step - loss: 8.3953e-04 - acc: 0.9999

Epoch 478/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 26us/step - loss: 0.0011 - acc: 0.9999

Epoch 479/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 9.0768e-04 - acc: 0.9999

Epoch 480/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 0.0010 - acc: 0.9999

Epoch 481/500
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 8.7743e-04 - acc: 0.9999

Epoch 482/500
100040/100040 [==============================]100040/100040 [===========

In [31]:
model.save('./model-23')

## Test data

In [19]:
test_dataset = pd.read_csv('test.csv', delimiter=',')
test_data = test_dataset.drop('id', axis=1)

In [20]:
test_labels = test_data.apply(calc_hand_label, axis=1)

In [21]:
test_data_with_distances = with_distances(test_data)
test_data_encoded = one_hot_encoded(test_data_with_distances, suits, encoder)
test_data_normalized = normalized(test_data_encoded, cards, mean_c, std_c)
test_data_normalized = normalized(test_data_normalized, distances, mean_d, std_d)

In [22]:
test_data_normalized.shape

(1000000, 30)

## Predictions

In [25]:
test_data_sample_predicted_10 = model.predict(np.array(test_data_normalized))
test_data_sample_predicted = np.array(list(map(np.argmax, test_data_sample_predicted_10)))

In [26]:
predictions_data = pd.concat([
    test_dataset,
    pd.DataFrame({'predictions': test_data_sample_predicted}),
    pd.DataFrame({'labels': test_labels}),
], axis=1)

In [27]:
wrong_answers = predictions_data[predictions_data['predictions'] != predictions_data['labels']]
wrong_answers.shape

(23, 13)

In [28]:
wrong_answers['labels'].value_counts()

4    16
9     3
0     3
5     1
Name: labels, dtype: int64

In [29]:
wrong_answers

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predictions,labels
26585,26586,4,12,4,11,2,2,4,13,2,10,4,0
50026,50027,4,11,1,13,3,1,2,10,3,12,0,4
50832,50833,2,12,1,10,2,1,1,13,4,11,0,4
128563,128564,4,2,3,12,2,13,4,11,3,10,4,0
140499,140500,2,13,1,11,2,1,3,10,3,12,0,4
183170,183171,1,13,1,11,3,12,3,10,3,1,0,4
195791,195792,1,13,1,12,3,11,3,10,1,1,0,4
203210,203211,2,10,1,12,3,1,1,11,2,13,0,4
236265,236266,3,13,1,10,4,12,4,11,2,1,0,4
285881,285882,3,11,1,12,4,10,4,13,1,1,0,4


In [30]:
wrong_answers.drop(['id', 'predictions', 'labels'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

26585     Q♠, J♠, 2♦, K♠, 10♦
50026     J♠, K♥, A♣, 10♦, Q♣
50832     Q♦, 10♥, A♦, K♥, J♠
128563    2♠, Q♣, K♦, J♠, 10♣
140499    K♦, J♥, A♦, 10♣, Q♣
183170    K♥, J♥, Q♣, 10♣, A♣
195791    K♥, Q♥, J♣, 10♣, A♥
203210    10♦, Q♥, A♣, J♥, K♦
236265    K♣, 10♥, Q♠, J♠, A♦
285881    J♣, Q♥, 10♠, K♠, A♥
293874    J♣, K♥, 10♣, Q♣, A♦
417823    J♥, K♥, Q♣, A♠, 10♦
424685    K♥, J♥, A♦, Q♣, 10♥
440230    A♠, J♠, 10♠, Q♠, K♠
492023    K♦, Q♦, A♣, 10♣, J♣
546426    2♣, 10♠, J♦, Q♣, K♦
588350    Q♠, A♠, K♠, J♠, 10♠
651415    J♠, K♥, A♦, 10♥, Q♠
706973    K♣, Q♥, J♣, 10♥, A♦
732871    K♠, Q♥, J♠, 10♣, A♦
755190     2♦, A♦, Q♦, J♦, K♦
794296    10♦, J♦, K♦, A♦, Q♦
846914    J♣, Q♥, 10♣, K♥, A♦
dtype: object